In [15]:
## Create Colab Notebook folder
from google.colab import drive
drive.mount('/gdrive')
!ln -s "/gdrive/My Drive/Colab Notebooks" "/content/Colab Notebooks"

# Put Colab in the context of the project
import os
# os.chdir allows you to change directories, like cd in the Terminal
os.chdir('/content/Colab Notebooks/AerialImageDataset')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [16]:
%cd /content/Colab Notebooks
!git clone https://github.com/JennyF-slb/solar_ai.git
%cd /content/Colab Notebooks/solar_ai
!pip install -r minimal_requirements.txt

/gdrive/My Drive/Colab Notebooks
fatal: destination path 'solar_ai' already exists and is not an empty directory.
/gdrive/My Drive/Colab Notebooks/solar_ai
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Import Libraries

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# 2. Seeding

In [ ]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

# 3. Hyperparameters

In [ ]:
batch_size = xx
lr = xx
epochs = xx 
height =  xx
width = xx

# 4. Set Data Path

In [ ]:
dataset_path = xx
files_dir = xx
model_file = xx
log_file = xx

# 5. Creating Folder

# 6. Data Pipeline

-- Read image and mask 

-- Define train and test dataset 

-- Load as numpy tensor 

-- make it to train, test dataloader 


# 7. Data Visualization / EDA

# 8. Data Augmentation - To handle imbalance class

-- VFlip, HFlip, Rotation, RandomCrop, ....(To research) 

-- Check if need to apply GAN / Diffusion model to generate synthetic data

# 9. Data Preparation for model training

-- Check need apply image patching with overlap

-- Define input image & label (match dimension)

-- Data normalization (check to apply global / local mean)

-- Define loader (train, test data loader)

# 10. Define Model (-- Need handle imbalance class)

## 10.1 Base Model

## 10.2 Unet

In [ ]:
## Convolution Layer

def conv2d_block (input_tensor, n_filters, kernel_size =3):
    x = input_tensor
    for i in range(2):
        x = tf.keras.layers.Conv2D(filters = n_filters,
                                   kernel_size = (kernel_size, kernel_size))(x)
        ## can add batch normalization to speed up training and make learning easier
        # x = tf. BatchNormalization
        x = tf.keras.layers.Activation('relu')(x)

    return x 

In [ ]:
## Encoder block

def encoder_block(inputs, n_filters, pool_size, dropout):
    f = conv2d_block(inputs, n_filters=n_filters)
    p = tf.keras.layers.MaxPooling2D(pool_size)(f)
    p = tf.keras.layers.Dropout(dropout)(p)
    
    return f,p

In [ ]:
## Calling encoder

def encoder(inputs):
    f1, p1 = encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3)
    f2, p2 = encoder_block(p1, n_filters=128, pool_size=(2,2), dropout=0.3)
    f3, p3 = encoder_block(p2, n_filters=256, pool_size=(2,2), dropout=0.3)
    f4, p4 = encoder_block(p3, n_filters=512, pool_size=(2,2), dropout=0.3)
    
    return p4, (f1, f2, f3, f4)

In [ ]:
def bottleneck (inputs):
    bottle_neck = conv2d_block(inputs, n_filters=1024)
    
    return bottle_neck

In [ ]:
## decoder block

def decoder_block (inputs, conv_output, n_filters, kernel_size, strides, dropout):
    u = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size, strides=strides, padding='same')(inputs)
    c = tf.keras.layers.concatenate([u, conv_output]) ## skip connection
    c = tf.keras.layers.Dropout(dropout)(c)
    c = conv2d_block(c, n_filters, kernel_size=3)
    
    return c

In [ ]:
## calling decoder

def decoder(inputs, convs):
    f1, f2, f3, f4 = convs
    
    c6 = decoder_block(inputs. f4, n_filters=512, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    c7 = decoder_block(c6, f3, n_filters=256, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    c8 = decoder_block(c7, f2, n_filters=128, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    c9 = decoder_block(c8, f1, n_filters=64, kernel_size=(3,3), strides=(2,2), dropout=0.3)
    
    outputs = tf.keras.layers.Conv2D(3,(1,1), activation='softmax')(c9)  ## 3 because we have 3 classes
    
    return outputs
    

In [ ]:
## unet model

def unet():
    inputs = tf.keras.layers.Input(shape=(128,128,3)) ## height x width x channel size
    encoder_output , convs =  encoder(input)
    
    bottle_neck = bottleneck(encoder_output)

    outputs = decoder(bottle_neck, convs)
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    
    return model

## 10.3 Segformer

# 11. Model Training (Cross Validation 5 fold)

# 12. Model Evalution (Loss Function)


# 13. Callback - to save the model checkpoint

# 14. Model Interpretation (Class Activation Map)

### 14.1 Gradient CAM, Salient Map



# 15. Model Prediction/ Inference